In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"
random_state = 281997

BGG Does not directly provide a way to list all the games it has in archive therefore we used a dump created by the community (2024-08-18).

# Dataset Generation
Our dataset is a corpus of reviews scrapped from the BGG API. <br /> 
In order to download the comments we make use of the ```bgg_corpus_service.py``` content.

## Subsample the data
We should limit the number of reviews, how many? Let's look at some case studies:

- Amazon Product Reviews
Size: Varies by category, but subsets of 5,000 to 20,000 reviews are common.
- Yelp Dataset
Size: Typically, 8,000 to 15,000 reviews are used in research for unsupervised aspect extraction.
- TripAdvisor Reviews
Size: Around 5,000 to 10,000 reviews in unsupervised experiments.

For unsupervised learning, 5,000–10,000 reviews is a reasonable starting point for recognizing 6 aspects. More reviews may improve diversity and robustness but come with increased computational costs.




In [2]:
import pandas as pd

corpus_file = "../data/corpus.csv"
sampled_corpus_file = "../data/corpus.sampled.csv"

In [3]:
og_data = pd.read_csv(corpus_file)
reviews_per_game = int(64000 / len(og_data.groupby(["game_id"]).count())) + 1

print(f"I have a total of {len(og_data.groupby(["game_id"]).count())} games with reviews. "
      f"We want to be ~64k reviews so we take {reviews_per_game} reviews per game.")

I have a total of 2220 games with reviews. We want to be ~64k reviews so we take 29 reviews per game.


In [4]:
# We start by using ~64k reviews (More robustness). This is before pre-processing which might reduce the total number of reviews later.
(
    og_data.groupby("game_id", group_keys=False)[og_data.columns]
    .apply(lambda x: x.sample(min(len(x), reviews_per_game), random_state=random_state))
    .to_csv(sampled_corpus_file, index=False)
)

Check distribution of games

In [5]:
data = pd.read_csv(sampled_corpus_file)
data

,comments,game_id
0,I like the 1986 version much more than the 199...,1
1,New edition. It's a great political game that...,1
2,Consider selling,1
3,"+ Interesting, complex, classic deep game. - T...",1
4,Just two incomplete plays. Not enough to give ...,1
...,...,...
64375,今天要给小动物们找个家！ 超主观点评： 1、主题好，画风可可爱爱，规则非常简单，适合推新。如...,414317
64376,Available on BGA,414317
64377,Played on BGA. Mostly solitaire abstract puzz...,414317
64378,2025 for Amanda?,414317


In [6]:
data.groupby(["game_id"]).count()
# Each of our games has the same representation then others. The "reviews" should be balanced across all games.
# We can now proceed to pre-process the data.

,comments
game_id,
1,29
3,29
5,29
10,29
11,29
...,...
397385,29
397598,29
400314,29


# Preprocessing
The downloaded information from the BGG API might not be informative, faulty or bloated with useless information. <br>
In order to avoid this we apply some pre-processing steps in order to filter out information we don't need, that may be entire records or some of the 
text inside a line.

During the process we already make the tokenization and stemming of the text using the ```spacy```


In [3]:
import warnings

# Some parts of torch that are used by Spacy are deprecated, we can ignore them 
# (The new 3.8 Spacy has some little issues, so we keep it like it is for now)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Using Spacy
To download the model and use it with spacy:
```
python -m spacy download en_core_web_sm
```

In [8]:
import spacy

model = spacy.load("en_core_web_sm")

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._

## PreProcessingService
Class that holds the process to clean the text and produce a stemmed corpus. <br/> This will then be persisted in a file to avoid re-processing the same data.

In [9]:
from pre_processing import PreProcessingService

ps = PreProcessingService()

In [10]:
demo_text = "This is a demo text. Isn't Root just an amazing game? I love it!"

### BGG noise removal
BGG comments can carry metadata such as images and some pseudo-html tags. <br>
To avoid processing those we simply remove them applying two regexes:

In [11]:
# As defined in the PreProcessingService
clean_tags_regex = r"(?i)\[(?P<tag>[A-Z]+)\].*?\[/\1\]"
keep_tag_content_regex = r"\[(?P<tag>[a-z]+)(=[^\]]+)?\](.*?)\[/\1\]"

In [12]:
ps.clean_text("This is a test for processing [IMG]https://cf.geekdo-static.com/mbs/mb_5855_0.gif[/IMG] as content")

'This is a test for processing  as content'

In [13]:
ps.clean_text("This is a test for processing [b=323]bold[/b] as content")

'This is a test for processing bold as content'

### Language detection
While it of course would be amazing to have a model with multiple languages support, we are focusing on English. <br>
To filter out foreign languages we use the ```langdetect``` library.

In [14]:
from fast_langdetect import detect

german_sentence = "Naja, ich finde die Siedler von Catan immer noch besser"
print(f"For the demo sentence: \"{demo_text}\" we detected: {detect(demo_text)['lang']}")
print(f"For the demo sentence: \"{german_sentence}\" we detected: {detect(german_sentence)['lang']}")

For the demo sentence: "This is a demo text. Isn't Root just an amazing game? I love it!" we detected: en
For the demo sentence: "Naja, ich finde die Siedler von Catan immer noch besser" we detected: de


### Tokenization and lemmatization
Using ```spacy``` we tokenize the text and then we lemmatize it. <br>

In [15]:
ps._make_text_lemmas(demo_text)  # (Should be considered private)

['demo', 'text', 'root', 'amazing', 'game', 'love']

### Remove too narrow texts
Comments (reviews) that are too short might not be informative. <br>
We already remove stopwords and punctuation, so we can filter out comments that are too short but we better set a reasonable threshold (not too high). This step is done by the PreProcessingService aswell.

In [16]:
ps.pre_process(demo_text)

'demo text root amazing game love'

## Batch Process

In [4]:
preprocessed_corpus_file: str = "../data/corpus.preprocessed.csv"

In [6]:
from pre_processing import pre_process_corpus

pre_process_corpus(sampled_corpus_file, preprocessed_corpus_file, False)

Pandas Apply:   0%|          | 0/64380 [00:00<?, ?it/s]

See how the dataset changed:

In [7]:
len(pd.read_csv(preprocessed_corpus_file))  # We lost 14k reviews but it is okay! (I expect to lose more)

50461

# Custom Dataset Definition
To train the model we require a way to get elements of our dataset. ```torch``` provides a way to do this by defining a custom ```Dataset``` class. <br>
This class and later loaded into a ```DataLoader``` that will provide the batches of data to the model.

In order to generate valid inputs for the model we have to give a numerical representation to our data. <br>
In order to do so we use a ```WordEmbedding``` model that will give us the dictionary of the recognized words (The embeddings will be generated inside the model). <br>

In [5]:
max_vocab_size = 16000
embedding_size = 128
target_embedding_model_file = "./../data/word-embeddings.model"

In [6]:
import core.utils as utils
import core.embeddings as embeddings

embeddings_model = embeddings.WordEmbedding(
    utils.LoadCorpusUtility(), max_vocab_size=max_vocab_size, embedding_size=embedding_size,
    target_model_file=target_embedding_model_file, corpus_file=preprocessed_corpus_file
)

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
# We require a vocabulary to map the words to indexes
embeddings_model.load_model()
embeddings_model.get_vocab()

vocabulary = embeddings_model.model.wv.key_to_index

INFO:gensim.utils:loading Word2Vec object from ../data/word-embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': '../data/word-embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from ../data/word-embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': '../data/word-embeddings.model', 'datetime': '2024-11-28T17:22:20.450083', 'gensim': '4.3.3', 'python': '3.12.3 (main, Nov  6 2024, 18:32:19) [GCC 13.2.0]', 'platform': 'Linux-6.8.0-49-generic-x86_64-with-glibc2.39', 'event': 'loaded'}


## PositiveNegativeCommentGeneratorDataset
Gives a sample and also returns some negative samples for contrastive learning. <br>


In [19]:
from core.dataset import PositiveNegativeCommentGeneratorDataset

ds = PositiveNegativeCommentGeneratorDataset("./../data/corpus.preprocessed.csv", vocabulary, 10)

Loading spacy model.
Loading dataset from file: ./../data/corpus.preprocessed.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/50461 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Padding sequences to max length (256).


KeyboardInterrupt: 

In [9]:
from torch.utils.data import DataLoader

lazy_dataloader = DataLoader(ds, batch_size=32, shuffle=True)

In [17]:
i = 11  # A random index to show content and 
print(f"Sentence at index {i} original text is: `{ds.get_text_item(i)}` (Look at [comments] property for the stripped down version)\n "
      f"It's numeric representation:\n {ds[i][0][0]}")

Sentence at index 11 original text is: `Fun, but a bit complex for my taste.` (Look at [comments] property for the stripped down version)
 It's numeric representation:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0

### Sequence length truncation
The model will be trained on sequences of fixed length. <br>
The chosen length must be reasonable, we can't just pad everything out for the same of it. <br>

We want that the top 95% of the reviews are not truncated. <br>

In [ ]:
# We have 137 of the 50461 total reviews that are bigger than 256 tokens.
# This is less than 1% of the total reviews. We can truncate.